In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd

In [5]:
# Login Info
cookies = {
    'bggusername': 'PatOs',
    'bggpassword': '1w9fv10td4gx2zyu9l8fp2rf7qpb2qlte'
}

In [6]:
url = 'https://boardgamegeek.com/browse/boardgame/page/21'

pages = requests.get(url, cookies=cookies)
pages.text

'<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t\t<script>\n\t\t\twindow.addEventListener( \'DOMContentLoaded\',  function() {\n\t\t\t\tvar width = document.documentElement.clientWidth || window.innerWidth;\n\t\t\t\tif (width < 960) {\n\t\t\t\t\tvar mvp = document.getElementById(\'vp\');\n\t\t\t\t\t// android debugging\n\t\t\t\t\tmvp.setAttribute(\'content\',\'width=960\');\n\t\t\t\t}\n\t\t\t});\n\t\t</script>\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Browse Board Games | BoardGameGeek</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://c

In [7]:
soup = BeautifulSoup(pages.text, 'html')
soup

<!DOCTYPE html>
<html lang="en-US" ng-app="GeekApp" ng-cloak="">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" id="vp" name="viewport"/>
<script>
			window.addEventListener( 'DOMContentLoaded',  function() {
				var width = document.documentElement.clientWidth || window.innerWidth;
				if (width < 960) {
					var mvp = document.getElementById('vp');
					// android debugging
					mvp.setAttribute('content','width=960');
				}
			});
		</script>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Browse Board Games | BoardGameGeek</title>
<link href="https://cf.geekdo-static.com/icons/touch-icon180.png" rel="apple-touch-icon"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="icon" type="image/ico"/>
<link href="/game-opensearch.xml" rel="search" title="BGG 

In [ ]:
# Quick RegExp to find the first number in the text
def get_number(x):
  x = str(x)
  return re.findall('[-+]?\d*\.*\d+', x)[0]

# takes in the rakings url and returns a dataframe of the information
def parse_rankings_page(url):

  # grab the html
  print('Getting page :', url, end='')
  pages = requests.get(url + str(url), cookies=cookies)

  # Check to see for an okay response
  if(pages.status_code != 200):
    print('Status code not 200')
    return None
  
  #convert to soup
  soup = BeautifulSoup(pages.text, 'html')

  # DataFrame for all the scraped information
  games = {
      'Game_Name':[],
      'Game_ID':[],
      'Rank':[],
      'Geek_Rating':[], 
      'Average_Rating':[], 
      'Number_of_Voters':[],
      'Description':[]
      }

  #find all the columns to do with BGG ratings
  ratings_soup = soup.find_all('td', class_= 'collection_bggrating')

  for x in range(0, len(ratings_soup), 3):
    games['Geek_Rating'].append(float(get_number(ratings_soup[x])))
    games['Average_Rating'].append(float(get_number(ratings_soup[x + 1])))
    games['Number_of_Voters'].append(int(get_number(ratings_soup[x + 2])))

  # find the game links and extract database id and game name
  links = soup.find_all('a', class_= 'primary', href=True)

  for link in links:
    games['Game_Name'].append(link.text)
    games['Game_ID'].append(int(get_number(link['href'])))

  # find short descriptions and release year if they're there
  filter = re.compile('CEcell_objectname*')
  descriptions = soup.find_all(id=filter)
  for description in descriptions:
    try:
      games['Description'].append(description.find('p', class_='smallefont dull').text[4:-3])
    except:
      games['Description'].append('')

  # find game rank
  ranks = soup.find_all('td', class_= 'collection_rank')
  for rank in ranks:
    games['Rank'].append(int(rank.text))
  
  df = pd.DataFrame(games)
  print('   Rows added:', len(df))

  return df

In [ ]:
# Test our our parser to make sure it correctly gets the information we want
pd.concat([parse_rankings_page('https://boardgamegeek.com/browse/boardgame/page/1'), 
          parse_rankings_page('https://boardgamegeek.com/browse/boardgame/page/2')])

Getting page : https://boardgamegeek.com/browse/boardgame/page/1   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/2   Rows added: 100


,Game_Name,Game_ID,Rank,Geek_Rating,Average_Rating,Number_of_Voters,Description
0,Brass: Birmingham,224517,1,8.426,8.62,37388,"Build networks, grow industries, and navigate ..."
1,Pandemic Legacy: Season 1,161936,2,8.395,8.54,50114,Mutating diseases are spreading around the wor...
2,Gloomhaven,174430,3,8.394,8.63,57589,Vanquish monsters with strategic cardplay. Ful...
3,Ark Nova,342942,4,8.295,8.54,28630,"Plan and build a modern, scientifically manage..."
4,Twilight Imperium: Fourth Edition,233078,5,8.241,8.62,20463,"Build an intergalactic empire through trade, r..."
...,...,...,...,...,...,...,...
95,Calico,283155,196,7.308,7.60,14951,"Sew a quilt, collect buttons, attract cats!"
96,Combat Commander: Europe,21050,197,7.308,7.95,5794,Recreate the European theater of World War II ...
97,Coimbra,245638,198,7.307,7.64,11103,Spread your influence across monasteries of Po...
98,Star Realms: Colony Wars,182631,199,7.306,7.81,7607,"Expand your fleet, defend your bases, and prev..."


In [ ]:
# scrape the first 100 pages of BGG rankings
# Save the information in a DataFrame
# We must be logged in to see past page 20 of the boadgame rankings
url = 'https://boardgamegeek.com/browse/boardgame/page/'
games = pd.DataFrame()

for x in range(1, 101):
  parsed_games = parse_rankings_page(url + str(x))
  time.sleep(3)

  games = pd.concat([games, parsed_games])

print('Scraping Done. Total rows:', len(games))

Getting page : https://boardgamegeek.com/browse/boardgame/page/1   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/2   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/3   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/4   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/5   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/6   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/7   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/8   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/9   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/10   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/11   Rows added: 100
Getting page : https://boardgamegeek.com/browse/boardgame/page/12   Rows added: 100
G

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
filepath = Path('/content/gdrive/MyDrive/DataWorks MD Talk/games.csv')
games.to_csv(filepath, index=False)  

In [ ]:
games

,Game_Name,Game_ID,Rank,Geek_Rating,Average_Rating,Number_of_Voters,Description
0,Brass: Birmingham,224517,1,8.426,8.62,37388,"Build networks, grow industries, and navigate ..."
1,Pandemic Legacy: Season 1,161936,2,8.395,8.54,50114,Mutating diseases are spreading around the wor...
2,Gloomhaven,174430,3,8.394,8.63,57589,Vanquish monsters with strategic cardplay. Ful...
3,Ark Nova,342942,4,8.295,8.54,28630,"Plan and build a modern, scientifically manage..."
4,Twilight Imperium: Fourth Edition,233078,5,8.241,8.62,20463,"Build an intergalactic empire through trade, r..."
...,...,...,...,...,...,...,...
95,Cartoon Network Crossover Crisis: Animation An...,221367,9996,5.563,7.01,81,
96,"Jours de Gloire Campagne IV: Allemagne 1813, d...",96061,9997,5.563,7.41,76,
97,Battles of Normandy: A Solitaire Wargame,356224,9998,5.563,8.32,60,British troops push through Normandy as part o...
98,Würfelwurst,118215,9999,5.563,6.14,215,
